In [4]:
# Imports

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [354]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_length, state=None):
        if state is None:
            self.state = np.random.randint(0, board_length, board_length)
        else:
            self.state = np.array(state)  # Ensure state is numpy array
        self.goal_fitness = int(((board_length ** 2) - board_length) / 2)
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Row conflicts
            count += (current_queen == remaining_queens).sum()
            
            # Diagonal conflicts
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            count += sum((remaining_queens == upper_diagonal) | 
                        (remaining_queens == lower_diagonal))
        
        return self.goal_fitness - count

    def is_goal(self):
        fitness = self.fitness()
        if fitness == self.goal_fitness:
            return True, self.state, fitness
        return False, None, fitness

In [373]:
# Genetic Algorithm


class GeneticAlgorithm:
    def __init__(self, board_size, population_size, mutation_probability, max_generations, initial_population=None):
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.max_generations = max_generations
        self.fitness_history = []
        self.best_solution = None
        self.best_fitness = float('-inf')
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        
        if initial_population is None:
            self.current_population = [EightQueensState(board_size) 
                                     for _ in range(population_size)]
        else:
            self.current_population = initial_population

    def calculate_population_fitness(self):
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()
            self.fitness_history.append(fitness)
            
            if fitness > self.best_fitness:
                self.best_fitness = fitness
                self.best_solution = individual.state.copy()
            
            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Add selection probability
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness if total_fitness > 0 else 1/len(population_data)
            
        return sorted(population_data, key=lambda x: x['fitness'], reverse=True)

    def select_parent(self, population_data):
        probabilities = [ind['selection_prob'] for ind in population_data]
        idx = np.random.choice(len(population_data), p=probabilities)
        return population_data[idx]['state']

    def crossover(self, parent1, parent2):
        crossover_point = np.random.randint(1, len(parent1))
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        return child1, child2

    def mutate(self, state):
        if np.random.random() < self.mutation_probability:
            idx = np.random.randint(len(state))
            new_value = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
            state[idx] = new_value
        return state

    def create_next_generation(self, population_data):
        next_generation = []
        
        # Elitism: Keep the best individual
        next_generation.append(EightQueensState(self.board_size, population_data[0]['state']))
        
        while len(next_generation) < self.population_size:
            parent1 = self.select_parent(population_data)
            parent2 = self.select_parent(population_data)
            
            child1, child2 = self.crossover(parent1, parent2)
            
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        # Trim if we added one too many children
        self.current_population = next_generation[:self.population_size]

    def run(self):
        for generation in range(self.max_generations):
            population_data = self.calculate_population_fitness()
            
            # Check if we've found a solution
            if population_data[0]['fitness'] == self.goal_fitness:
                print(f"Solution found in generation {generation}")
                return self.fitness_history, self.best_solution
            
            self.create_next_generation(population_data)
        
        print("Max generations reached without finding solution")
        return self.fitness_history, self.best_solution

In [374]:
# Plot GA

def plot_ga(solution, fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(0, 255, 0, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Final Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        height = 700,
        width=1500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [ ]:
# Run

board_size = 8
population_size = 100
mutation_probability = 0.1
max_generations = 10000

ga = GeneticAlgorithm(
    board_size=board_size,
    population_size=population_size,
    mutation_probability=mutation_probability,
    max_generations=max_generations
)

fitness_history, best_solution = ga.run()

plot_ga(28, fitness_history)